In [56]:
import random
import pandas as pd
import numpy as np
import pickle
from deap import base, creator, tools, algorithms
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import warnings

warnings.filterwarnings("ignore")

with open(r"/Users/taejin/Desktop/데분 학회/급행버스/api 딕셔너리/getStationByRoute.pickle", 'rb') as file:
    getStationByRoute = pickle.load(file)

with open(r'/Users/taejin/Downloads/getStationByPosList.pickle', 'rb') as f:
    # 객체를 로드합니다.
    getStationByPosList = pickle.load(f)

with open(r'/Users/taejin/Desktop/데분 학회/급행버스/api 딕셔너리/노선별_정류장Kmeans정리.pickle', 'rb') as f:
    # 객체를 로드합니다.
    station_info = pickle.load(f)

od = pd.read_csv('/Users/taejin/Desktop/데분 학회/급행버스/od_weekday_2.csv')


### 1. 유전 알고리즘에 적용하기 위한 데이터 프레임 함수

In [57]:
def travel_time_by_station(getStationByRoute, route_name):
    df = getStationByRoute[route_name] # 피클에서 노선 정보 불러오기
    df['arsId'] = df['arsId'].astype(int)
    df = df[['busRouteNm', 'arsId', 'stationNm', 'seq', 'sectSpd', 'fullSectDist']] #필요한 행만 불러오기
    df['travelTime'] = df['fullSectDist'].astype(int) * 0.06/ 30 #travel time 열 추가 (30km, 0.06 분으로 변환하기 위한 값 60min/1000km)
    df['seq'] = df['seq'].astype(int) # 정류장 순서 타입 int로 변경

    return df

def passenger_onoff_num(od, route_name):
    od['승객수'] = od['승객수'].astype(int)
    od = od[od['노선명'] == route_name]
    boarding_data = od.groupby('승차_정류장순번')['승객수'].sum().reset_index()
    boarding_data.columns = ['정류장순번', '승차수']

    # Group by 하차_정류장순번 and sum 승객수
    alighting_data = od.groupby('하차_정류장순번')['승객수'].sum().reset_index()
    alighting_data.columns = ['정류장순번', '하차수']

    # Merge the boarding and alighting data on 정류장순번
    result = pd.merge(boarding_data, alighting_data, on='정류장순번', how='outer').fillna(0)
    result['승객총합'] = result['승차수'] + result['하차수']
    result['소요시간'] = result[['승차수', '하차수']].max(axis=1) * 2.3 /60 #승하차시 소요시간(분)
    #버스 10대/시간, 운행시간 18시간으로 나누어 버스 한대당 으로 변환
    result[['승차수', '하차수', '승객총합', '소요시간']] = result[['승차수', '하차수', '승객총합', '소요시간']]/180
    
    return result


def station_kmeans_result(station_df):
    scaler = MinMaxScaler()
    station_df[['dist', 'bus_cnt', '승하차총승객수']] = scaler.fit_transform(station_df[['dist', 'bus_cnt', '승하차총승객수']])

    # K-means 군집화
    kmeans = KMeans(n_clusters=4, random_state=42)
    station_df['cluster'] = kmeans.fit_predict(station_df[['dist', 'bus_cnt', '승하차총승객수']])

    # 높은 승객 수, 많은 버스 수, 짧은 거리의 정류장 선택
    selected_cluster = station_df.groupby('cluster')['승하차총승객수'].mean().idxmax(axis=0)

    # 급행 버스 정류장 선택
    express_stations = station_df[station_df['cluster'] == selected_cluster]

    return express_stations

def gene_algorithm_data(route_name): #유전 알고리즘을 위한 정류장별 데이터
    df1 = travel_time_by_station(getStationByRoute, route_name)
    df2 = passenger_onoff_num(od, route_name)
    df3 = station_kmeans_result(station_info[route_name])
    
    df3['cluster'] = 1
    df3 = dict(zip(df3['arsId'].astype(int), df3['cluster']))
    # display(df1)
    # display(df2)

    df1 = pd.merge(df1, df2, left_on = 'seq', right_on = '정류장순번', how = 'left')

    df1['cluster'] = df1['arsId'].map(df3)
    df1['cluster'].replace(np.nan, 0, inplace=True)

    df1 = df1[['arsId', 'seq', 'travelTime', '승차수', '하차수', '승객총합', '소요시간','cluster']]

    return df1

### 2. 유전 알고리즘 구현

#### 유전 알고리즘 목적 함수 구성

In [58]:
# 가중치 파라미터 (조정해야 하는 값)
a = 0.1
b = 2

# 정차 여부에 따른 승객 수 및 노선 소요 시간 계산 함수
def total_passenger(individual, df):
    df['cluster'] = individual
    passenger = (df['승객총합'] * df['cluster']).sum()
    return passenger

def travel_time(individual, df):
    df['cluster'] = individual
    # 구간 통행속도
    travel_time = df['travelTime'].sum()
    # 가감속 시간
    acc_time = (df['cluster'] * 11.6/60).sum()
    # 승하차시간
    onoff_time = (df['cluster'] * df['소요시간']).sum()
    travel_time = travel_time + acc_time + onoff_time
    return travel_time

# 적합도 함수
def fitness(individual):
    total_pass = total_passenger(individual, df)
    total_time = travel_time(individual, df)
    fit_value = + b * total_time - a * total_pass + sum(individual)
    return fit_value,

# 개체를 초기화할 때 고정된 유전자를 설정하고 1의 개수를 제한하는 함수
def initIndividual(icls, fixed_indices, fixed_values, num_stops):
    individual = [0] * num_stops
    num_ones = int(num_stops * 0.3)
    
    # 고정된 유전자 설정
    for idx, value in zip(fixed_indices, fixed_values):
        individual[idx] = value
        if value == 1:
            num_ones -= 1
    
    # 나머지 유전자 설정
    available_indices = [i for i in range(num_stops) if i not in fixed_indices]
    ones_indices = random.sample(available_indices, num_ones)
    
    for idx in ones_indices:
        individual[idx] = 1
    
    return icls(individual)

#### 유전 알고리즘

In [59]:
def gene_algorithm(df, fixed_indices, fixed_values):
    num_stops = len(df)

    # 유전 알고리즘 설정
    if not hasattr(creator, 'FitnessMin'):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # 적합도 클래스 생성

    if not hasattr(creator, 'Individual'):
        creator.create("Individual", list, fitness=creator.FitnessMin)  # 개체 클래스 생성


    # 유전 알고리즘 도구상자
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)  # 개체 유전자 초기화 속성
    toolbox.register("individual", initIndividual, creator.Individual, fixed_indices, fixed_values, num_stops)  # 개체 속성
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)  # 모집단 속성

    toolbox.register("evaluate", fitness)  # 평가 속성, fitness(적합도 함수) 입력 필요
    toolbox.register("mate", tools.cxOnePoint)  # 교차 속성

    # 고정된 유전자를 반영한 돌연변이 연산 정의
    def mutFlipBitFixed(individual, indpb, fixed_indices):
        for i in range(len(individual)):
            if i not in fixed_indices and random.random() < indpb:
                individual[i] = type(individual[i])(not individual[i])
        return individual,

    # 도구 상자에 돌연변이 연산 등록
    toolbox.register("mutate", mutFlipBitFixed, indpb=0.05, fixed_indices=fixed_indices)

    # 교차 연산 수정
    def cxOnePointFixed(ind1, ind2, fixed_indices):
        size = min(len(ind1), len(ind2))
        cxpoint = random.randint(1, size - 1)
        for i in range(size):
            if i not in fixed_indices and i < cxpoint:
                ind1[i], ind2[i] = ind2[i], ind1[i]
        return ind1, ind2

    toolbox.register("select", tools.selRoulette)

    # 초기 개체군 생성
    population = toolbox.population(n=50)

    # 유전 알고리즘 실행
    result_population = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=1000,
                                            stats=None, halloffame=None, verbose=False)

    # 최적 해 찾기
    best_individual = tools.selBest(result_population[0], k=1)[0]
    # print("Best Individual: ", best_individual)
    # print("Fitness: ", fitness(best_individual)[0])

    return best_individual

### 3. 유전 알고리즘 결과 추출

In [67]:
route_names = ['143','160','172']
result_dict={}

for name in route_names:

    df = gene_algorithm_data(name)
    
    # 고정된 유전자를 설정
    fixed_indices = list(df[df['cluster'] == 1].index)  
    fixed_values = [1 for _ in range(len(fixed_indices))]   # 고정된 유전자의 값
    
    result = gene_algorithm(df, fixed_indices, fixed_values)
    exist = travel_time([1 for _ in range(len(df))], df)
    new = travel_time(result, df)

    print('            기존         급행')
    print(f'{name}: {exist/60}, {new/60}')
    print(f'정차 정류장 개수 : {sum(result)}')
    print(f'기존 정류장 개수 : {len(result)}')
    print(f'절감 시간 퍼센트 : { round(((exist/60)-(new/60)) * 100 / (exist/60), 2) }')
    print("Best Individual: ", result)
    print('----------------------------------------------------------------------------------\n')

    result_df = pd.DataFrame({
                                'exist' : [1 for _ in range(len(df))], 
                                'new' : result})
    result_dict[name] = result_df
    

            기존         급행
143: 2.554608487654321, 2.253803549382716
정차 정류장 개수 : 47
기존 정류장 개수 : 113
절감 시간 퍼센트 : 11.77
Best Individual:  [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1]
----------------------------------------------------------------------------------

            기존         급행
160: 2.8847828703703704, 2.599221913580247
정차 정류장 개수 : 54
기존 정류장 개수 : 119
절감 시간 퍼센트 : 9.9
Best Individual:  [1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 

### 4. 시각화를 위한 결과 내보내기

In [70]:
import pickle

with open(r"/Users/taejin/Desktop/데분 학회/급행버스/api 딕셔너리/getStationByRoute.pickle", 'rb') as file:
    getStationByRoute = pickle.load(file)

In [77]:
for name in route_names:
    result_dict[name].to_csv(f'/Users/taejin/Desktop/데분 학회/급행버스/{name}번 버스.csv')
    data = getStationByRoute[name]
    data = data[['arsId', 'gpsX', 'gpsY', 'seq', 'stationNm', 'direction']]
    data[['exist', 'new']] = result_dict[name][['exist', 'new']]
    data['routeNm'] = name
    data.to_csv(f'/Users/taejin/Desktop/데분 학회/급행버스/{name}번 버스.csv')

### 5. 143번 노선 활용 이동시 걸리는 시간 예시 계산

In [96]:
df_143 = gene_algorithm_data('143')
df_143 = df_143[(df_143['seq']>=8) & (df_143['seq']<=19)]
exist_143 = travel_time([1 for _ in range(len(df_143))], df_143)
new_143 = travel_time([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], df_143)
print(exist_143, new_143)

17.848499999999998 15.07162037037037


In [88]:
[1 for _ in range(len(df_143))]

[1, 1, 1, 1, 1, 1, 1, 1, 1]

In [89]:
df_143['cluster']

18    1
19    1
20    1
21    1
22    1
23    1
24    1
25    1
26    1
Name: cluster, dtype: int64